Import packages needed

In [1]:
import suite2p
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from natsort import natsorted
from multiprocessing import Pool
from joblib import Parallel, delayed
from pathlib import Path
import gc
gc.collect()
import importlib
import sys
from pathlib import Path
# Add the directory containing your script to the Python path
sys.path.append(r'C:\Users\User\Project-SLMonlineControl\PYsubfun')
#sys.path.append(r'C:\Users\zhangl33\Projects\Project-SLMonlineControl\PYsubfun')

# Now you can import your script as a module
import FastBin_Suite2p as FBS

In [2]:
#importlib.reload(FBS)
# Pre-set parameters, and load the only bin files in the pre-set path folde
ConfigFolder=r'C:\Users\User\Project-SLMonlineControl\config'
#ConfigFolder=r'C:\Users\zhangl33\Projects\Project-SLMonlineControl\config'
ops0, confSet=FBS.configLoad(ConfigFolder,'SLMsetting.yml','ops3Planes.npy')
SaveFolder=FBS.suite2pInitiate(ops0,ConfigFolder)
binFile = glob.glob(ops0['save_path0'] + '/TSeries*-001.bin')    ##<<<<<<<<<<<<<<<<<<<<<<<<<< Noted that there supposed be only one Bin Data file
#Load Data
print(binFile)
rawBin, FramePerPlane, TotalFrameNeed=FBS.LoadBin(binFile,ops0)
rawBin.shape

Processed data would be saved inE:\LuSLMOnlineTest\SL0855-Emx1G6CII-AAV9CAMKII\03042025\suite2p
File found: C:\Users\User\Project-SLMonlineControl\config\SLMsetting.yml
Copying to: E:\LuSLMOnlineTest\SL0855-Emx1G6CII-AAV9CAMKII\03042025\CurrentSLMsetting.yml
Current SLMsetting copied successfully!
Folder already exists.
['E:\\LuSLMOnlineTest\\SL0855-Emx1G6CII-AAV9CAMKII\\03042025\\TSeries-03042025-1059-001.bin']


(24600, 512, 512)

In [3]:
ops0['spatial_scale']=2
ops0['threshold_scaling']=1.6
ops0['denoise']=1
ops0['tau']=1.5

In [4]:
confSet

{'save_path0': 'E:\\LuSLMOnlineTest\\SL0855-Emx1G6CII-AAV9CAMKII\\03042025\\',
 'SLM_Pixels_X': 512,
 'SLM_Pixels_Y': 512,
 'scan_Z': 195.75,
 'ETL': '-50 0 50',
 'IsSpiral': 'True',
 'SpiralSizeUM': 15,
 'SpiralRevolution': 3,
 'Repetition': 5,
 'Duration': 20,
 'InterPointDelay': 30,
 'UncagingLaserPower': '1.35 1.45 1.55 1.65',
 'InitialDelay': 22.1,
 'NumNonTarget': 20,
 'NumNTperTrial': 9,
 'NumTrial': 5,
 'RadiusAvoidParam': 3,
 'nchannels': 1,
 'fs': 6.9,
 'tau': 1.5,
 'Ziteration': 11,
 'ZRepetition': 34}

Suite2p processing with no motion correction; Noted following defining a function to process a plane of data; Next session, parallel processing for each plane is excuted. Much time is saved. 
Alternatively, run this process with Step2-OnlineBinSuite2p.ipynb

In [5]:
# Define the number of parallel processes; 
# Noted that current binning movie in chuns of length 06; increase this parameter could decrease the number of frames within a chunks, using less memory
num_processes=1              ## <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# Create a list of plane indices
plane_indices = range(0,ops0['nplanes'])
print(plane_indices)
ops1=ops0
Parallel(n_jobs=num_processes)(delayed(FBS.process_plane)(plane_idx, rawBin, SaveFolder, ops1) for plane_idx in plane_indices)

# Combine multiple plane data
#FBS.CombinePlanes(SaveFolder,ops0)
FBS.CombinePlanes(SaveFolder,ops0)

range(0, 3)
Processing plane 0
Binning movie in chunks of length 10
Binned movie of size [820,512,512] created in 8.20 sec.
Binned movie denoised (for cell detection only) in 16.25 sec.
NOTE: FORCED spatial scale ~12 pixels, time epochs 1.00, threshold 16.00 
0 ROIs, score=204.09
Detected 118 ROIs, 14.71 sec
After removing overlaps, 118 ROIs remain
added enhanced mean image
Masks created, 0.75 sec.


c:\users\user\suite2p\suite2p\extraction\extract.py:125: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  Fi[n] = np.dot(data[:, cell_ipix[n]], cell_lam[n])


Extracted fluorescence from 118 ROIs in 8200 frames, 16.58 sec.
Extracted fluorescence from 118 ROIs in 8200 frames, 10.24 sec.
['skew', 'npix_norm', 'compact']
Processing plane 1
Binning movie in chunks of length 10
Binned movie of size [820,512,512] created in 7.66 sec.
Binned movie denoised (for cell detection only) in 16.14 sec.
NOTE: FORCED spatial scale ~12 pixels, time epochs 1.00, threshold 16.00 
0 ROIs, score=206.72
Detected 162 ROIs, 15.47 sec
After removing overlaps, 158 ROIs remain
added enhanced mean image
Masks created, 0.75 sec.
Extracted fluorescence from 158 ROIs in 8200 frames, 10.43 sec.
Extracted fluorescence from 158 ROIs in 8200 frames, 10.41 sec.
['skew', 'npix_norm', 'compact']
Processing plane 2
Binning movie in chunks of length 10
Binned movie of size [820,512,512] created in 7.71 sec.
Binned movie denoised (for cell detection only) in 15.59 sec.
NOTE: FORCED spatial scale ~12 pixels, time epochs 1.00, threshold 16.00 
0 ROIs, score=264.94
Detected 185 ROIs, 

Run Suite2p gui from command line, load combined data above, do necessary mannual correction. Saved the data.

Once above mannual correction is done; update the result in each plane

In [7]:
FBS.PostMannual(SaveFolder,ops0)

263 of units including 81 identified cells in combined planes
